# Description

The notebook contains full versions of DAG output checks that compare the prod system output vs that of the prod simulation run.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging

import core.config as cconfig
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import oms as oms

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-af9e003c-b951-4ed5-ba93-17bc808e1d21.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.2' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask4301_rollback_history_lookback_for_C3a'
  hash='7fbc421e9'
  # Last commits:
    * 7fbc421e9 saggese  Another solution to the end of line issue                         (   4 hours ago) Fri May 19 10:17:08 2023  (HEAD -> CmTask4301_rollback_history_lookback_for_C3a, origin/master, origin/HEAD, master)
    * 0cebe1765 saggese  Finally find a solution to the end of line issue                  (   4 hours ago) Fri May 19 0

# Build the reconciliation config

In [5]:
# Get config from env when running the notebook via the `run_notebook.py` script, e.g.,
# in the system reconciliation flow.
config = cconfig.get_config_from_env()
if config:
    _LOG.info("Using config from env vars")
else:
    _LOG.info("Using hardwired config")
    # Specify the config directly when running the notebook manually.
    # Below is just an example.
    dst_root_dir = "/shared_data/ecs/preprod/prod_reconciliation/"
    dag_builder_name = "C3a"
    start_timestamp_as_str = "20230517_131000"
    end_timestamp_as_str = "20230518_130500"
    run_mode = "paper_trading"
    mode = "scheduled"
    config_list = oms.build_reconciliation_configs(
        dst_root_dir,
        dag_builder_name,
        start_timestamp_as_str,
        end_timestamp_as_str,
        run_mode,
        mode,
    )
    config = config_list[0]
print(config)

INFO  Using hardwired config
INFO  run_date='20230517'
INFO  Using run_date=20230517
INFO  dst_root_dir='/shared_data/ecs/preprod/prod_reconciliation/', dag_builder_name='C3a', start_timestamp_as_str='20230517_131000', run_mode='paper_trading'
INFO  run_date='20230517'
INFO  target_dir='/shared_data/ecs/preprod/prod_reconciliation/C3a/20230517/paper_trading'
INFO  system_log_dir='system_log_dir.scheduled.20230517_131000.20230518_130500'
meta: 
  date_str: 20230517
  asset_class: crypto
  run_tca: False
  bar_duration: 5T
system_log_path: 
  prod: /shared_data/ecs/preprod/prod_reconciliation/C3a/20230517/paper_trading/prod/system_log_dir.scheduled.20230517_131000.20230518_130500
  sim: /shared_data/ecs/preprod/prod_reconciliation/C3a/20230517/paper_trading/simulation/system_log_dir
system_config_func_as_str: dataflow_orange.system.Cx.get_Cx_system_config_template_instance('C3a')
research_forecast_evaluator_from_prices: 
  init: 
    price_col: vwap
    prediction_col: feature
    volati

# Specify data to load

In [6]:
# The dict points to `system_log_dir` for different experiments.
system_log_path_dict = dict(config["system_log_path"].to_dict())

In [7]:
# This dict points to `system_log_dir/dag/node_io/node_io.data` for different experiments.
data_type = "dag_data"
dag_path_dict = oms.get_system_log_paths(system_log_path_dict, data_type)
dag_path_dict

{'prod': '/shared_data/ecs/preprod/prod_reconciliation/C3a/20230517/paper_trading/prod/system_log_dir.scheduled.20230517_131000.20230518_130500/dag/node_io/node_io.data',
 'sim': '/shared_data/ecs/preprod/prod_reconciliation/C3a/20230517/paper_trading/simulation/system_log_dir/dag/node_io/node_io.data'}

# DAG io

## Load

In [8]:
# Get DAG node names.
dag_node_names = oms.get_dag_node_names(dag_path_dict["prod"])
_LOG.info(
    "First node='%s' / Last node='%s'", dag_node_names[0], dag_node_names[-1]
)

INFO  First node='predict.0.read_data' / Last node='predict.5.process_forecasts'


In [9]:
# Get timestamps for the last DAG node.
dag_node_timestamps = oms.get_dag_node_timestamps(
    dag_path_dict["prod"], dag_node_names[-1], as_timestamp=True
)
_LOG.info(
    "First timestamp='%s'/ Last timestamp='%s'",
    dag_node_timestamps[0][0],
    dag_node_timestamps[-1][0],
)

INFO  First timestamp='2023-05-17 09:10:00-04:00'/ Last timestamp='2023-05-18 09:05:00-04:00'


In [10]:
# Get DAG output for the last node and the last timestamp.
dag_df_prod = oms.load_dag_outputs(dag_path_dict["prod"], dag_node_names[-1])
dag_df_sim = oms.load_dag_outputs(dag_path_dict["sim"], dag_node_names[-1])
_LOG.info("Output of the last node:\n")
hpandas.df_to_str(dag_df_prod, num_rows=5, log_level=logging.INFO)

INFO  Output of the last node:



feature                                                                                                                                                                                                                                                                          close_vol                                                                                                                                                                                                                                                                         garman_klass_vol                                                                                                                                                                                                                                                                         parkinson_vol                                                                                                                                                                                                                                                                              close                                                                                                                                                                                                                                                                                cmf                                                                                                                                                                                                                                                                               high                                                                                                                                                                                                                                                                                low                                                                                                                                                                                                                                                                               open                                                                                                                                                                                                                                                                               twap                                                                                                                                                                                                                                                                             volume                                                                                                                                                                                                                                                                                vwap                                                                                                                                                                                                                                                                              
                          1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573 1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 2384892553 2425308589 2476706208 2484635488 2540896331 2601760471 2683705052 3065029174 3401245610 4516629366 5115052901 5118394986 6051632686 8717633868 8968126878 9872743573       1030828978 1464553467 1467591036 1528092593 1776791608 1891737434 1966583502 2099673105 2237530510 238489

## Check DAG io self-consistency

In [11]:
# TODO(Grisha): pass the value via config.
diff_threshold = 1e-3
compare_dfs_kwargs = {
    # TODO(Nina): CmTask4387 "DAG self-consistency check fails when
    # `history_lookback=15T` for C3a".
    "compare_nans": False,
    "diff_mode": "pct_change",
    "assert_diff_threshold": None,
}

In [12]:
# Run for all timestamps.
bar_timestamp = "all"
# Compare DAG output at T with itself at time T-1.
oms.check_dag_output_self_consistency(
    dag_path_dict["prod"],
    dag_node_names[-1],
    bar_timestamp,
    trading_freq=config["meta"]["bar_duration"],
    diff_threshold=diff_threshold,
    **compare_dfs_kwargs,
)

## Compare DAG io (prod vs sim)

In [ ]:
# Run for all nodes and all timestamps.
bar_timestamp = "all"
node_name = "all"
oms.compare_dag_outputs(
    dag_path_dict,
    node_name,
    bar_timestamp,
    diff_threshold=diff_threshold,
    **compare_dfs_kwargs,
)